# Chef Infrastructure TCO Analysis

This notebook provides a comprehensive Total Cost of Ownership (TCO) analysis for Chef infrastructure estates. It includes:

1. **Data Loading & Validation** - Load organization data and assess data quality
2. **Benchmark Comparison** - Compare metrics against industry standards
3. **Hypothesis Testing** - Test key assumptions about costs
4. **TCO Calculation** - Calculate complete cost breakdown
5. **Scenario Analysis** - Compare migration options
6. **Visualizations** - Generate executive-ready charts

---

## Setup and Dependencies

In [ ]:
# Install required packages if needed
# !pip install pyyaml pandas numpy matplotlib seaborn plotly

import sys
import json
from pathlib import Path
from datetime import datetime

# Add calculator to path
sys.path.insert(0, str(Path('../calculator').resolve()))

import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Try to import plotly for interactive charts
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("Plotly not available. Install with: pip install plotly")

# Import our calculator
from tco_calculator import (
    TCOCalculator, OrganizationData, create_sample_data,
    load_from_yaml, BENCHMARKS
)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("Setup complete!")

---

## 1. Data Loading & Validation

In [ ]:
# Load data - choose one option:

# Option 1: Use sample data (200K nodes, 90K cookbooks)
data = create_sample_data()

# Option 2: Load from YAML file
# data = load_from_yaml('../analysis/sample_data.yaml')

# Create calculator instance
calculator = TCOCalculator(data)

print("Data Loaded Successfully!")
print(f"\nInfrastructure Summary:")
print(f"  Total Nodes: {data.infrastructure.total_managed_nodes:,}")
print(f"  Chef Servers: {data.infrastructure.chef_server_count}")
print(f"\nCookbook Summary:")
print(f"  Total Cookbooks: {data.cookbooks.total_cookbooks:,}")
print(f"  Active Cookbooks: {data.cookbooks.active_cookbooks:,}")
print(f"\nTeam Summary:")
print(f"  Dedicated Engineers: {data.team.dedicated_engineers}")
print(f"  Part-time Contributors: {data.team.part_time_contributors}")

### Data Quality Assessment

In [ ]:
# Calculate derived metrics for validation
total_fte = data.team.dedicated_engineers + (data.team.part_time_contributors * data.team.part_time_allocation_pct / 100)
cookbook_ratio = (data.cookbooks.active_cookbooks / data.infrastructure.total_managed_nodes) * 1000
cookbooks_per_fte = data.cookbooks.active_cookbooks / total_fte if total_fte > 0 else 0

validation_checks = {
    "Node count > 0": data.infrastructure.total_managed_nodes > 0,
    "Active cookbooks > 0": data.cookbooks.active_cookbooks > 0,
    "Cookbook tiers sum correctly": (
        data.cookbooks.tier1_simple + 
        data.cookbooks.tier2_standard + 
        data.cookbooks.tier3_complex == data.cookbooks.active_cookbooks
    ),
    "Salary in reasonable range": 80000 <= data.team.average_salary <= 300000,
    "Benefits multiplier reasonable": 1.2 <= data.team.benefits_multiplier <= 1.8,
    "License cost matches nodes": abs(
        data.licensing.annual_license_cost - 
        (data.infrastructure.total_managed_nodes * data.licensing.negotiated_rate_per_node)
    ) < data.licensing.annual_license_cost * 0.1,  # Within 10%
}

print("Data Validation Results:")
print("-" * 50)
all_passed = True
for check, passed in validation_checks.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"  {status}: {check}")
    if not passed:
        all_passed = False

print("-" * 50)
if all_passed:
    print("All validation checks passed!")
else:
    print("⚠️  Some validation checks failed. Review data before proceeding.")

---

## 2. Benchmark Comparison

In [ ]:
# Calculate health metrics
health = calculator.calculate_health_metrics()

# Create benchmark comparison dataframe
benchmarks_data = [
    {
        "Metric": "Cookbooks per 1,000 Nodes",
        "Your Value": health.cookbook_ratio,
        "Healthy Range": "5 - 25",
        "Warning Range": "25 - 100",
        "Critical Threshold": "> 100",
        "Status": "🔴 Critical" if health.cookbook_ratio > 100 else ("🟡 Warning" if health.cookbook_ratio > 25 else "🟢 Healthy")
    },
    {
        "Metric": "Cookbooks per FTE",
        "Your Value": health.cookbooks_per_fte,
        "Healthy Range": "150 - 250",
        "Warning Range": "75 - 150",
        "Critical Threshold": "< 75",
        "Status": "🔴 Critical" if health.cookbooks_per_fte < 75 else ("🟡 Warning" if health.cookbooks_per_fte < 150 else "🟢 Healthy")
    },
    {
        "Metric": "Technical Debt Multiplier",
        "Your Value": health.debt_multiplier,
        "Healthy Range": "1.0",
        "Warning Range": "1.1 - 1.5",
        "Critical Threshold": "> 1.5",
        "Status": "🔴 Critical" if health.debt_multiplier > 1.5 else ("🟡 Warning" if health.debt_multiplier > 1.0 else "🟢 Healthy")
    },
]

benchmark_df = pd.DataFrame(benchmarks_data)
print("\n📊 BENCHMARK COMPARISON")
print("=" * 80)
display(benchmark_df)

print(f"\n🏥 Overall Health Score: {health.health_score.upper()}")
if health.issues:
    print("\n⚠️  Issues Identified:")
    for issue in health.issues:
        print(f"   • {issue}")

In [ ]:
# Visualize benchmark comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Chart 1: Cookbook Ratio Gauge
ax1 = axes[0]
categories = ['Healthy\n(<25)', 'Warning\n(25-100)', 'Critical\n(>100)', 'Your Value']
values = [25, 75, 100, health.cookbook_ratio]
colors = ['#2ecc71', '#f39c12', '#e74c3c', '#3498db']
bars = ax1.bar(categories, values, color=colors, edgecolor='white', linewidth=2)
ax1.set_ylabel('Cookbooks per 1,000 Nodes')
ax1.set_title('Cookbook Ratio vs Benchmarks')
ax1.axhline(y=25, color='#2ecc71', linestyle='--', alpha=0.7, label='Healthy threshold')
ax1.axhline(y=100, color='#e74c3c', linestyle='--', alpha=0.7, label='Critical threshold')
for bar, val in zip(bars, values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val:.0f}', 
             ha='center', va='bottom', fontweight='bold')

# Chart 2: Cookbooks per FTE
ax2 = axes[1]
categories = ['Critical\n(<75)', 'Warning\n(75-150)', 'Healthy\n(>150)', 'Your Value']
values = [75, 112, 200, health.cookbooks_per_fte]
colors = ['#e74c3c', '#f39c12', '#2ecc71', '#3498db']
bars = ax2.bar(categories, values, color=colors, edgecolor='white', linewidth=2)
ax2.set_ylabel('Cookbooks per FTE')
ax2.set_title('Team Efficiency vs Benchmarks')
for bar, val in zip(bars, values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val:.0f}', 
             ha='center', va='bottom', fontweight='bold')

# Chart 3: Debt Multiplier
ax3 = axes[2]
multipliers = ['1.0x\n(<25)', '1.1x\n(25-50)', '1.25x\n(50-100)', '1.5x\n(100-250)', '2.0x\n(250-500)', '2.5x\n(>500)']
mult_values = [1.0, 1.1, 1.25, 1.5, 2.0, 2.5]
colors = ['#2ecc71', '#27ae60', '#f39c12', '#e67e22', '#e74c3c', '#c0392b']
bars = ax3.bar(multipliers, mult_values, color=colors, edgecolor='white', linewidth=2)
ax3.axhline(y=health.debt_multiplier, color='#3498db', linestyle='-', linewidth=3, label=f'Your multiplier: {health.debt_multiplier:.2f}x')
ax3.set_ylabel('Debt Multiplier')
ax3.set_title('Technical Debt Multiplier Scale')
ax3.legend(loc='upper left')

plt.tight_layout()
plt.savefig('visualizations/benchmark_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("📈 Chart saved to: visualizations/benchmark_comparison.png")

---

## 3. Hypothesis Testing

In [ ]:
# Test Hypothesis H1: Cookbook sprawl increases maintenance cost non-linearly

print("🔬 HYPOTHESIS TESTING")
print("=" * 70)

print("\n📋 H1: Cookbook sprawl increases maintenance cost non-linearly")
print("-" * 60)

# Calculate what costs would be at different cookbook ratios
ratios = [10, 25, 50, 100, 250, 500, health.cookbook_ratio]
multipliers = []
for ratio in ratios:
    for threshold, mult in sorted(BENCHMARKS["debt_multipliers"].items()):
        if ratio <= threshold:
            multipliers.append(mult)
            break

# Base labor cost (without multiplier)
base_labor = (
    data.team.dedicated_engineers * data.team.average_salary * data.team.benefits_multiplier +
    data.team.part_time_contributors * (data.team.part_time_allocation_pct/100) * 
    data.team.average_salary * data.team.benefits_multiplier
)

costs_at_ratios = [base_labor * m for m in multipliers]

print(f"  Current cookbook ratio: {health.cookbook_ratio:.1f} per 1,000 nodes")
print(f"  Current debt multiplier: {health.debt_multiplier:.2f}x")
print(f"  Base labor cost: ${base_labor:,.0f}")
print(f"  Actual labor cost (with debt): ${base_labor * health.debt_multiplier:,.0f}")
print(f"  Technical debt tax: ${base_labor * (health.debt_multiplier - 1):,.0f}")

# Test result
if health.cookbook_ratio > 100 and health.debt_multiplier > 1.25:
    print("\n  ✅ H1 SUPPORTED: High cookbook ratio correlates with elevated debt multiplier")
else:
    print("\n  ❓ H1 INCONCLUSIVE: Need more data points to confirm relationship")

In [ ]:
# Test Hypothesis H2: Technical debt ratio correlates with incident frequency

print("\n📋 H2: Technical debt ratio correlates with incident frequency")
print("-" * 60)

cookbook_node_ratio = data.cookbooks.active_cookbooks / data.infrastructure.total_managed_nodes
monthly_incidents = data.incidents.monthly_incidents
incidents_per_1k_nodes = (monthly_incidents / data.infrastructure.total_managed_nodes) * 1000

print(f"  Cookbook-to-node ratio: {cookbook_node_ratio:.4f} ({cookbook_node_ratio*100:.2f}%)")
print(f"  Monthly incidents: {monthly_incidents}")
print(f"  Incidents per 1,000 nodes/month: {incidents_per_1k_nodes:.3f}")

# Expected incidents based on industry (typically 0.05-0.1 per 1K nodes/month for healthy)
expected_healthy = 0.05
expected_degraded = 0.15

if cookbook_node_ratio > 0.1 and incidents_per_1k_nodes > expected_healthy:
    print(f"\n  ✅ H2 SUPPORTED: High cookbook ratio ({cookbook_node_ratio:.2f}) correlates with")
    print(f"     elevated incident rate ({incidents_per_1k_nodes:.3f} vs {expected_healthy} healthy baseline)")
else:
    print("\n  ❓ H2 INCONCLUSIVE: Incident rate is within acceptable range")

In [ ]:
# Test Hypothesis H4: Modern alternatives reduce TCO by 30-50%

print("\n📋 H4: Modern alternatives reduce TCO by 30-50% at scale")
print("-" * 60)

# Calculate current TCO
tco = calculator.calculate_tco()
current_annual = tco.total_annual_tco
current_3yr = current_annual * 3

# Calculate scenarios
scenarios = {}
for platform in ['ansible', 'kubernetes', 'terraform']:
    scenarios[platform] = calculator.calculate_scenario(platform)

print(f"  Current 3-year TCO: ${current_3yr:,.0f}")
print("\n  Alternative 3-year TCO:")

h4_supported = False
for platform, scenario in scenarios.items():
    savings = current_3yr - scenario.three_year_total
    savings_pct = (savings / current_3yr) * 100
    print(f"    {platform.title()}: ${scenario.three_year_total:,.0f} (savings: {savings_pct:.1f}%)")
    if savings_pct >= 30:
        h4_supported = True

if h4_supported:
    print("\n  ✅ H4 SUPPORTED: At least one alternative offers 30%+ savings")
else:
    print("\n  ❌ H4 NOT SUPPORTED: No alternative meets 30% savings threshold")

---

## 4. TCO Calculation

In [ ]:
# Get complete TCO breakdown
tco = calculator.calculate_tco()
per_unit = calculator.calculate_per_unit_costs()

print("💰 ANNUAL TCO BREAKDOWN")
print("=" * 50)

# Create breakdown dataframe
breakdown_data = [
    {"Category": "Licensing", "Cost": tco.licensing_cost, "Type": "Direct"},
    {"Category": "Infrastructure", "Cost": tco.infrastructure_cost, "Type": "Direct"},
    {"Category": "Platform Labor", "Cost": tco.platform_labor_cost, "Type": "Labor"},
    {"Category": "Distributed Labor", "Cost": tco.distributed_labor_cost, "Type": "Labor"},
    {"Category": "Incident Response", "Cost": tco.incident_cost, "Type": "Labor"},
    {"Category": "Technical Debt Tax", "Cost": tco.technical_debt_tax, "Type": "Hidden"},
    {"Category": "Training", "Cost": tco.training_cost, "Type": "Other"},
    {"Category": "Contractors", "Cost": tco.contractor_cost, "Type": "Other"},
    {"Category": "Opportunity Cost", "Cost": tco.opportunity_cost, "Type": "Hidden"},
]

breakdown_df = pd.DataFrame(breakdown_data)
breakdown_df['Percentage'] = (breakdown_df['Cost'] / tco.total_annual_tco * 100).round(1)
breakdown_df['Cost_Formatted'] = breakdown_df['Cost'].apply(lambda x: f"${x:,.0f}")

display(breakdown_df[['Category', 'Cost_Formatted', 'Percentage', 'Type']])

print(f"\n{'─' * 50}")
print(f"TOTAL ANNUAL TCO: ${tco.total_annual_tco:,.0f}")
print(f"{'─' * 50}")

print(f"\n📊 Per-Unit Costs:")
print(f"  Per Node: ${per_unit['per_node']:,.2f}")
print(f"  Per Cookbook: ${per_unit['per_cookbook']:,.2f}")
print(f"  Per FTE: ${per_unit['per_fte']:,.0f}")

In [ ]:
# Visualize TCO breakdown
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart - Cost breakdown
ax1 = axes[0]
costs = breakdown_df['Cost'].values
labels = breakdown_df['Category'].values
colors = plt.cm.Set3(np.linspace(0, 1, len(costs)))

# Only show labels for significant categories
def make_autopct(values):
    def my_autopct(pct):
        return f'{pct:.1f}%' if pct > 3 else ''
    return my_autopct

wedges, texts, autotexts = ax1.pie(
    costs, labels=None, autopct=make_autopct(costs),
    colors=colors, startangle=90, pctdistance=0.75
)
ax1.legend(wedges, labels, title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
ax1.set_title(f'Annual TCO Breakdown\nTotal: ${tco.total_annual_tco/1e6:.1f}M', fontsize=14, fontweight='bold')

# Bar chart - By type
ax2 = axes[1]
type_totals = breakdown_df.groupby('Type')['Cost'].sum().sort_values(ascending=True)
colors = {'Direct': '#3498db', 'Labor': '#2ecc71', 'Hidden': '#e74c3c', 'Other': '#9b59b6'}
bar_colors = [colors[t] for t in type_totals.index]
bars = ax2.barh(type_totals.index, type_totals.values / 1e6, color=bar_colors, edgecolor='white', linewidth=2)
ax2.set_xlabel('Cost (Millions USD)')
ax2.set_title('Costs by Category Type', fontsize=14, fontweight='bold')
for bar in bars:
    width = bar.get_width()
    ax2.text(width + 0.1, bar.get_y() + bar.get_height()/2, f'${width:.1f}M',
             ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('visualizations/tco_breakdown.png', dpi=150, bbox_inches='tight')
plt.show()
print("📈 Chart saved to: visualizations/tco_breakdown.png")

In [ ]:
# Waterfall chart for TCO
fig, ax = plt.subplots(figsize=(14, 7))

categories = list(breakdown_df['Category']) + ['TOTAL']
values = list(breakdown_df['Cost']) + [tco.total_annual_tco]

# Calculate cumulative for waterfall
cumulative = np.cumsum([0] + values[:-1])

# Create waterfall effect
for i, (cat, val, cum) in enumerate(zip(categories[:-1], values[:-1], cumulative)):
    ax.bar(i, val / 1e6, bottom=cum / 1e6, color='#3498db', edgecolor='white', linewidth=2)
    # Add value label
    ax.text(i, (cum + val/2) / 1e6, f'${val/1e6:.1f}M', ha='center', va='center', 
            color='white', fontweight='bold', fontsize=9)

# Total bar
ax.bar(len(categories)-1, tco.total_annual_tco / 1e6, color='#2c3e50', edgecolor='white', linewidth=2)
ax.text(len(categories)-1, tco.total_annual_tco / 1e6 / 2, f'${tco.total_annual_tco/1e6:.1f}M', 
        ha='center', va='center', color='white', fontweight='bold', fontsize=11)

ax.set_xticks(range(len(categories)))
ax.set_xticklabels(categories, rotation=45, ha='right')
ax.set_ylabel('Cost (Millions USD)')
ax.set_title('TCO Waterfall Analysis', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('visualizations/tco_waterfall.png', dpi=150, bbox_inches='tight')
plt.show()
print("📈 Chart saved to: visualizations/tco_waterfall.png")

---

## 5. Scenario Analysis

In [ ]:
# Calculate all migration scenarios
print("🔄 MIGRATION SCENARIO ANALYSIS")
print("=" * 70)

scenarios = {}
for platform in ['ansible', 'kubernetes', 'terraform', 'puppet']:
    scenarios[platform] = calculator.calculate_scenario(platform)

# Create comparison dataframe
scenario_data = []
for platform, s in scenarios.items():
    scenario_data.append({
        "Platform": platform.title(),
        "Migration Cost": f"${s.migration_cost/1e6:.1f}M",
        "Year 1": f"${s.year1_cost/1e6:.1f}M",
        "Year 2": f"${s.year2_cost/1e6:.1f}M",
        "Year 3": f"${s.year3_cost/1e6:.1f}M",
        "3-Year Total": f"${s.three_year_total/1e6:.1f}M",
        "Breakeven": f"{s.breakeven_months:.0f} mo" if s.breakeven_months else "N/A",
        "NPV": f"${s.npv_3year/1e6:.1f}M",
        "Risk": s.risk_score.upper()
    })

# Add current state
current_3yr = tco.total_annual_tco * 3
scenario_data.insert(0, {
    "Platform": "Current (Chef)",
    "Migration Cost": "$0",
    "Year 1": f"${tco.total_annual_tco/1e6:.1f}M",
    "Year 2": f"${tco.total_annual_tco*1.05/1e6:.1f}M",
    "Year 3": f"${tco.total_annual_tco*1.1/1e6:.1f}M",
    "3-Year Total": f"${current_3yr*1.05/1e6:.1f}M",
    "Breakeven": "Baseline",
    "NPV": "Baseline",
    "Risk": "LOW"
})

scenario_df = pd.DataFrame(scenario_data)
display(scenario_df)

In [ ]:
# Visualize scenario comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Chart 1: 3-Year Total Comparison
ax1 = axes[0]
platforms = ['Current\n(Chef)', 'Ansible', 'Kubernetes', 'Terraform', 'Puppet']
totals = [
    current_3yr * 1.05,  # With 5% growth
    scenarios['ansible'].three_year_total,
    scenarios['kubernetes'].three_year_total,
    scenarios['terraform'].three_year_total,
    scenarios['puppet'].three_year_total,
]
colors = ['#e74c3c', '#3498db', '#2ecc71', '#9b59b6', '#f39c12']
bars = ax1.bar(platforms, [t/1e6 for t in totals], color=colors, edgecolor='white', linewidth=2)
ax1.set_ylabel('3-Year TCO (Millions USD)')
ax1.set_title('3-Year Total Cost Comparison', fontsize=14, fontweight='bold')
ax1.axhline(y=totals[0]/1e6, color='#e74c3c', linestyle='--', alpha=0.5, label='Current baseline')
for bar, total in zip(bars, totals):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'${total/1e6:.1f}M',
             ha='center', va='bottom', fontweight='bold')

# Chart 2: NPV Comparison
ax2 = axes[1]
npvs = [
    0,  # Baseline
    scenarios['ansible'].npv_3year,
    scenarios['kubernetes'].npv_3year,
    scenarios['terraform'].npv_3year,
    scenarios['puppet'].npv_3year,
]
bar_colors = ['#95a5a6' if n <= 0 else '#2ecc71' for n in npvs]
bars = ax2.bar(platforms, [n/1e6 for n in npvs], color=bar_colors, edgecolor='white', linewidth=2)
ax2.set_ylabel('3-Year NPV (Millions USD)')
ax2.set_title('Net Present Value of Migration', fontsize=14, fontweight='bold')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=1)
for bar, npv in zip(bars, npvs):
    offset = 0.5 if npv >= 0 else -1.5
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + offset, f'${npv/1e6:.1f}M',
             ha='center', va='bottom' if npv >= 0 else 'top', fontweight='bold')

plt.tight_layout()
plt.savefig('visualizations/scenario_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("📈 Chart saved to: visualizations/scenario_comparison.png")

In [ ]:
# Year-over-year comparison
fig, ax = plt.subplots(figsize=(12, 6))

years = ['Year 1', 'Year 2', 'Year 3']
x = np.arange(len(years))
width = 0.15

# Current state (with 5% annual increase)
current_costs = [tco.total_annual_tco, tco.total_annual_tco * 1.05, tco.total_annual_tco * 1.1]

bars1 = ax.bar(x - 2*width, [c/1e6 for c in current_costs], width, label='Current (Chef)', color='#e74c3c')
bars2 = ax.bar(x - width, [scenarios['ansible'].year1_cost/1e6, scenarios['ansible'].year2_cost/1e6, scenarios['ansible'].year3_cost/1e6], width, label='Ansible', color='#3498db')
bars3 = ax.bar(x, [scenarios['kubernetes'].year1_cost/1e6, scenarios['kubernetes'].year2_cost/1e6, scenarios['kubernetes'].year3_cost/1e6], width, label='Kubernetes', color='#2ecc71')
bars4 = ax.bar(x + width, [scenarios['terraform'].year1_cost/1e6, scenarios['terraform'].year2_cost/1e6, scenarios['terraform'].year3_cost/1e6], width, label='Terraform', color='#9b59b6')
bars5 = ax.bar(x + 2*width, [scenarios['puppet'].year1_cost/1e6, scenarios['puppet'].year2_cost/1e6, scenarios['puppet'].year3_cost/1e6], width, label='Puppet', color='#f39c12')

ax.set_xlabel('Year')
ax.set_ylabel('Annual Cost (Millions USD)')
ax.set_title('Year-over-Year Cost Comparison by Scenario', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(years)
ax.legend(loc='upper right')

plt.tight_layout()
plt.savefig('visualizations/yoy_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("📈 Chart saved to: visualizations/yoy_comparison.png")

---

## 6. Sensitivity Analysis

In [ ]:
# Sensitivity analysis - vary key parameters
print("📊 SENSITIVITY ANALYSIS")
print("=" * 70)

base_tco = tco.total_annual_tco

# Test different variations
sensitivity_results = []

# Vary license cost ±20%
for factor in [0.8, 0.9, 1.0, 1.1, 1.2]:
    adjusted_license = tco.licensing_cost * factor
    adjusted_total = base_tco - tco.licensing_cost + adjusted_license
    sensitivity_results.append({
        "Parameter": "License Cost",
        "Variation": f"{(factor-1)*100:+.0f}%",
        "Adjusted TCO": adjusted_total,
        "Impact": adjusted_total - base_tco
    })

# Vary labor cost ±15%
labor_base = tco.platform_labor_cost + tco.distributed_labor_cost
for factor in [0.85, 0.925, 1.0, 1.075, 1.15]:
    adjusted_labor = labor_base * factor
    adjusted_total = base_tco - labor_base + adjusted_labor
    sensitivity_results.append({
        "Parameter": "Labor Cost",
        "Variation": f"{(factor-1)*100:+.0f}%",
        "Adjusted TCO": adjusted_total,
        "Impact": adjusted_total - base_tco
    })

# Vary debt multiplier
for mult in [1.0, 1.25, 1.5, 2.0, 2.5]:
    adjusted_debt_tax = labor_base * (mult - 1)
    adjusted_total = base_tco - tco.technical_debt_tax + adjusted_debt_tax
    sensitivity_results.append({
        "Parameter": "Debt Multiplier",
        "Variation": f"{mult:.2f}x",
        "Adjusted TCO": adjusted_total,
        "Impact": adjusted_total - base_tco
    })

sensitivity_df = pd.DataFrame(sensitivity_results)
sensitivity_df['Impact_Formatted'] = sensitivity_df['Impact'].apply(
    lambda x: f"{'+' if x >= 0 else ''}{x/1e6:.2f}M"
)
sensitivity_df['TCO_Formatted'] = sensitivity_df['Adjusted TCO'].apply(lambda x: f"${x/1e6:.1f}M")

display(sensitivity_df[['Parameter', 'Variation', 'TCO_Formatted', 'Impact_Formatted']])

In [ ]:
# Tornado chart for sensitivity
fig, ax = plt.subplots(figsize=(12, 6))

# Calculate impact ranges for each parameter
parameters = ['License Cost (±20%)', 'Labor Cost (±15%)', 'Debt Multiplier (1.0-2.5x)']

# Get min and max for each parameter type
license_impacts = [r['Impact'] for r in sensitivity_results if r['Parameter'] == 'License Cost']
labor_impacts = [r['Impact'] for r in sensitivity_results if r['Parameter'] == 'Labor Cost']
debt_impacts = [r['Impact'] for r in sensitivity_results if r['Parameter'] == 'Debt Multiplier']

low_impacts = [min(license_impacts)/1e6, min(labor_impacts)/1e6, min(debt_impacts)/1e6]
high_impacts = [max(license_impacts)/1e6, max(labor_impacts)/1e6, max(debt_impacts)/1e6]

y_pos = np.arange(len(parameters))

# Create horizontal bars
for i, (param, low, high) in enumerate(zip(parameters, low_impacts, high_impacts)):
    ax.barh(i, high - low, left=low, color='#3498db', alpha=0.7, edgecolor='white', linewidth=2)
    ax.text(low - 0.5, i, f'${low:.1f}M', ha='right', va='center', fontsize=10)
    ax.text(high + 0.5, i, f'+${high:.1f}M', ha='left', va='center', fontsize=10)

ax.axvline(x=0, color='black', linestyle='-', linewidth=2)
ax.set_yticks(y_pos)
ax.set_yticklabels(parameters)
ax.set_xlabel('Impact on Annual TCO (Millions USD)')
ax.set_title('Sensitivity Analysis - TCO Impact by Parameter', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('visualizations/sensitivity_tornado.png', dpi=150, bbox_inches='tight')
plt.show()
print("📈 Chart saved to: visualizations/sensitivity_tornado.png")

---

## 7. Executive Summary & Recommendations

In [ ]:
# Generate final report
report = calculator.generate_report()

print("📋 EXECUTIVE SUMMARY")
print("=" * 70)

print(f"""
CURRENT STATE ASSESSMENT
────────────────────────────────────────────────────────────────────────
Infrastructure:    {report['summary']['total_nodes']:,} managed nodes
Cookbooks:         {report['summary']['active_cookbooks']:,} active cookbooks
Annual TCO:        ${report['summary']['annual_tco']:,.0f}
Per-Node Cost:     ${report['summary']['per_node_cost']:,.2f}
Per-Cookbook Cost: ${report['summary']['per_cookbook_cost']:,.2f}
Health Score:      {report['summary']['health_score'].upper()}

KEY FINDINGS
────────────────────────────────────────────────────────────────────────
• Cookbook ratio of {health.cookbook_ratio:.0f} per 1,000 nodes is {'CRITICAL' if health.cookbook_ratio > 100 else 'elevated'}
  (Industry best practice: <25 per 1,000 nodes)

• Technical debt multiplier of {health.debt_multiplier:.2f}x is costing 
  ${tco.technical_debt_tax:,.0f}/year in additional labor overhead

• {'Labor costs represent ' + str(round(tco.labor_costs/tco.total_annual_tco*100)) + '% of TCO - primary optimization target'}

RECOMMENDATIONS
────────────────────────────────────────────────────────────────────────
""")

for i, rec in enumerate(report['recommendations'], 1):
    print(f"{i}. {rec}\n")

# Find best scenario
best = max(report['scenarios'].items(), key=lambda x: x[1]['npv_3year'])
print(f"""
RECOMMENDED PATH FORWARD
────────────────────────────────────────────────────────────────────────
Based on the analysis, the recommended approach is: {best[0].upper()}

• 3-Year NPV: ${best[1]['npv_3year']:,.0f}
• Breakeven: {best[1]['breakeven_months']:.0f} months
• Risk Level: {best[1]['risk_score'].upper()}
• 3-Year Total Cost: ${best[1]['three_year_total']:,.0f}
• Savings vs Current: ${(current_3yr*1.05 - best[1]['three_year_total']):,.0f} (3-year)
""")

In [ ]:
# Export report to JSON
output_path = Path('visualizations/tco_report.json')
with open(output_path, 'w') as f:
    json.dump(report, f, indent=2, default=str)
print(f"\n📁 Full report exported to: {output_path}")

# Summary table for export
summary_df = pd.DataFrame([
    {"Metric": "Total Managed Nodes", "Value": f"{report['summary']['total_nodes']:,}"},
    {"Metric": "Active Cookbooks", "Value": f"{report['summary']['active_cookbooks']:,}"},
    {"Metric": "Annual TCO", "Value": f"${report['summary']['annual_tco']:,.0f}"},
    {"Metric": "Cost per Node", "Value": f"${report['summary']['per_node_cost']:,.2f}"},
    {"Metric": "Cost per Cookbook", "Value": f"${report['summary']['per_cookbook_cost']:,.2f}"},
    {"Metric": "Health Score", "Value": report['summary']['health_score'].upper()},
    {"Metric": "Cookbook Ratio (per 1K nodes)", "Value": f"{health.cookbook_ratio:.1f}"},
    {"Metric": "Debt Multiplier", "Value": f"{health.debt_multiplier:.2f}x"},
    {"Metric": "Recommended Platform", "Value": best[0].title()},
    {"Metric": "3-Year Potential Savings", "Value": f"${(current_3yr*1.05 - best[1]['three_year_total']):,.0f}"},
])

summary_df.to_csv('visualizations/executive_summary.csv', index=False)
print(f"📁 Executive summary exported to: visualizations/executive_summary.csv")

---

## Analysis Complete

All visualizations have been saved to the `visualizations/` directory:
- `benchmark_comparison.png` - Health metrics vs industry benchmarks
- `tco_breakdown.png` - Annual cost breakdown
- `tco_waterfall.png` - Waterfall view of costs
- `scenario_comparison.png` - Migration scenario comparison
- `yoy_comparison.png` - Year-over-year costs by scenario
- `sensitivity_tornado.png` - Sensitivity analysis
- `tco_report.json` - Full analysis data
- `executive_summary.csv` - Key metrics for reporting